# Fibre projector

Demonstrating project and voxelize capability of the fibre projector.

*Author: Vedrana Andersen Dahl (vand@dtu.dk)*


In [ ]:
import os
import fibre_packer as fp
import matplotlib.pyplot as plt

In [ ]:
# Parameters 
domain_radius = 70  # Domain radius
fibre_radius_mean = 10  # Mean fibre radius
fibre_radius_sigma = 0.1 * fibre_radius_mean  # Standard deviation of fibre radius
number_slices = 20 # Number of slices to generate
iters = 100
fibre_volume_fraction = 10
misalignment = 'none'


In [ ]:
fib = fp.from_fvf(domain_radius, fibre_volume_fraction, fibre_radius_mean, fibre_radius_sigma)
fib.initialize_start_slice()
losses = fib.optimize_slice_points('start', iters=iters, delta=0.5, k=1)
fib.initialize_end_slice(misalignment)
fib.rotate_bundle((0, 0), domain_radius, 2)
losses = fib.optimize_slice_points('end', iters=iters, delta=0.5, k=1)
fib.interpolate_configuration(number_slices)
losses = fib.optimize_configuration(iters=iters)


In [ ]:
fib.animate_slices()

In [ ]:
fib.show_3D_configuration()

In [ ]:
thetas = 45
side = 50
projections = fib.project(thetas=thetas, bins=side, new_z=side)

fig, ax = plt.subplots(5, 9, figsize=(15, 10))
for p, a in zip(projections, ax.flatten()):
    a.imshow(p, cmap='gray')
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(5, 10, figsize=(15, 10))
ax = ax.flatten()
for i in range(50):
     ax[i].imshow(projections[:, i, :].T)
plt.tight_layout()
plt.show()

In [ ]:
voxelization = fib.voxelize(pixels=side, new_z=side)

fig, ax = plt.subplots(5, 10, figsize=(15, 10))
for v, a in zip(voxelization, ax.flatten()):
    a.imshow(v, cmap='gray')
plt.tight_layout()
plt.show()

In [ ]:
from skimage.transform import iradon
import numpy as np

def reconstruct(projections, thetas, side=None):
    if side is None:
        side = projections.shape[2]
    if type(thetas) is int:
        thetas = np.arange(thetas) * 180 / thetas
    volume = np.empty((projections.shape[1], side, side))
    for i in range(projections.shape[1]):
        sinogram = projections[:, i, :].T
        fbp = iradon(sinogram.numpy(), theta=thetas, filter_name='ramp', output_size=side)
        volume[i] = np.flip(fbp, 0)
    return volume

reconstruction = reconstruct(projections, thetas=thetas, side=side)
fig, ax = plt.subplots(5, 10, figsize = (15, 10))
for r, a in zip(reconstruction, ax.flatten()):
    a.imshow(r, cmap='gray')
plt.tight_layout()
plt.show()